### Polecenie od klienta:
*"Mamy co prawda dodatkowe benefity dla naszych najlepszych klientów, ale może dałoby
się ustalić kto potencjalnie jest skłonny wydawać u nas więcej?”*

### Zadanie biznesowe
Sugerowanie klientów, którzy mogą wrócić do serwisu.

### Zadanie modelowania
Model regresyjny, szacujący prawdopodobieństwo powrotu klienta do serwisu.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
iteration_path = "iteration_3/"
deliveries_path = "../data/" + iteration_path + "raw/deliveries.jsonl"
products_path = "../data/" + iteration_path + "raw/products.jsonl"
sessions_path = "../data/" + iteration_path + "raw/sessions.jsonl"
users_path = "../data/" + iteration_path + "raw/users.jsonl"

In [3]:
deliveries_data = pd.read_json(deliveries_path, lines=True)
products_data = pd.read_json(products_path, lines=True)
sessions_data = pd.read_json(sessions_path, lines=True)
users_data = pd.read_json(users_path, lines=True)

In [83]:
deliveries_data

,purchase_id,purchase_timestamp,delivery_timestamp,delivery_company
0,20001,2021-05-19T07:02:36,2021-05-21T07:04:43,360
1,20002,2021-02-27T20:13:02,2021-03-02T20:15:54,516
2,20003,2021-04-25T13:44:28,2021-04-29T13:45:25,620
3,20004,2021-09-12T10:03:56,2021-09-14T10:07:12,516
4,20005,2021-07-10T16:12:54,2021-07-15T16:16:25,516
...,...,...,...,...
3608,23609,2021-11-05T01:26:21,2021-11-07T01:31:12,620
3609,23610,2021-07-05T18:23:31,2021-07-07T18:27:35,516
3610,23611,2021-07-20T17:15:10,2021-07-23T17:17:26,516
3611,23612,2021-11-06T05:53:32,2021-11-10T05:54:08,620


In [84]:
products_data

,product_id,product_name,category_path,price
0,1001,Telefon Siemens Gigaset DA310,Telefony i akcesoria;Telefony stacjonarne,58.97
1,1002,Kyocera FS-1135MFP,Komputery;Drukarki i skanery;Biurowe urządzeni...,2048.50
2,1003,Kyocera FS-3640MFP,Komputery;Drukarki i skanery;Biurowe urządzeni...,7639.00
3,1004,Fallout 3 (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,49.99
4,1005,Szalone Króliki Na żywo i w kolorze (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,49.99
...,...,...,...,...
314,1315,Jabra Talk,Telefony i akcesoria;Akcesoria telefoniczne;Ze...,54.99
315,1316,Plantronics Voyager Legend,Telefony i akcesoria;Akcesoria telefoniczne;Ze...,249.00
316,1317,Plantronics Savi W740,Telefony i akcesoria;Akcesoria telefoniczne;Ze...,1303.96
317,1318,Plantronics Savi W710,Sprzęt RTV;Audio;Słuchawki,553.00


In [5]:
sessions_data

,session_id,timestamp,user_id,product_id,event_type,offered_discount,purchase_id
0,124,2021-05-19 06:57:15,102,1277,VIEW_PRODUCT,5,NaN
1,124,2021-05-19 06:59:15,102,1276,VIEW_PRODUCT,5,NaN
2,124,2021-05-19 07:02:36,102,1276,BUY_PRODUCT,5,20001.0
3,125,2021-04-23 13:46:03,102,1284,VIEW_PRODUCT,20,NaN
4,125,2021-04-23 13:50:10,102,1292,VIEW_PRODUCT,20,NaN
...,...,...,...,...,...,...,...
24569,6881,2021-11-03 05:28:10,301,1073,VIEW_PRODUCT,5,NaN
24570,6881,2021-11-03 05:33:08,301,1201,VIEW_PRODUCT,5,NaN
24571,6881,2021-11-03 05:36:58,301,1072,VIEW_PRODUCT,5,NaN
24572,6881,2021-11-03 05:37:32,301,1222,VIEW_PRODUCT,5,NaN


In [85]:
users_data

,user_id,name,city,street
0,102,Arkadiusz Hejna,Wrocław,ul. Jana 32
1,103,Oskar Skotarczak,Warszawa,pl. Wrzosowa 595
2,104,Maksymilian Fik,Gdynia,al. Podwale 62
3,105,Fryderyk Hajdukiewicz,Gdynia,al. Narcyzowa 81/19
4,106,Marcelina Kinder,Wrocław,pl. Irysowa 472
...,...,...,...,...
195,297,Olga Myrcha,Kraków,plac Listopada 83/33
196,298,Nikodem Świgoń,Warszawa,ul. Złota 61/12
197,299,Gaja Wojtarowicz,Gdynia,ul. Baczynskiego 97
198,300,Inga Przewoźny,Radom,ulica Wysoka 33


In [6]:
sessions_data = sessions_data.sort_values(by=['timestamp'])
sessions_data['timestamp_date'] = sessions_data['timestamp'].apply(lambda x: x.date())
sessions_data['timestamp_week'] = sessions_data['timestamp'].apply(lambda x: x.week)
sessions_data['timestamp_month'] = sessions_data['timestamp'].apply(lambda x: x.month)
sessions_data['timestamp_quarter'] = sessions_data['timestamp'].apply(lambda x: x.quarter)

Podział na zbiór testowy i treningowy

In [7]:
sample_month_data = sessions_data[sessions_data.timestamp_month == 9]
train_data = sessions_data[sessions_data.timestamp_quarter < 4]
test_data = sessions_data[sessions_data.timestamp_quarter == 4]

Przykład join-a

In [32]:
pd.merge(sample_month_data, products_data, on="product_id").sort_values(by=['timestamp'])

,session_id,timestamp,user_id,product_id,event_type,offered_discount,purchase_id,timestamp_week,timestamp_month,timestamp_quarter,timestamp_date,product_name,category_path,price
0,2246,2021-09-01 02:00:24,158,1233,VIEW_PRODUCT,0,NaN,35,9,3,2021-09-01,Manta DVD064,Sprzęt RTV;Video;Odtwarzacze DVD,109.00
1,2246,2021-09-01 02:04:10,158,1233,BUY_PRODUCT,0,21170.0,35,9,3,2021-09-01,Manta DVD064,Sprzęt RTV;Video;Odtwarzacze DVD,109.00
76,198,2021-09-01 02:25:19,103,1072,VIEW_PRODUCT,15,NaN,35,9,3,2021-09-01,Jabra Freeway,Telefony i akcesoria;Akcesoria telefoniczne;Ze...,299.00
117,198,2021-09-01 02:28:27,103,1201,VIEW_PRODUCT,15,NaN,35,9,3,2021-09-01,Jabra Speak 410,Telefony i akcesoria;Akcesoria telefoniczne;Ze...,327.47
118,198,2021-09-01 02:33:16,103,1201,BUY_PRODUCT,15,20039.0,35,9,3,2021-09-01,Jabra Speak 410,Telefony i akcesoria;Akcesoria telefoniczne;Ze...,327.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2107,336,2021-09-30 23:03:20,106,1030,VIEW_PRODUCT,0,NaN,39,9,3,2021-09-30,LCD Iiyama B2280WSD,Komputery;Monitory;Monitory LCD,739.00
2108,336,2021-09-30 23:05:46,106,1030,BUY_PRODUCT,0,20116.0,39,9,3,2021-09-30,LCD Iiyama B2280WSD,Komputery;Monitory;Monitory LCD,739.00
2911,6769,2021-09-30 23:42:01,298,1276,VIEW_PRODUCT,0,NaN,39,9,3,2021-09-30,Apple iPad mini 64GB,Komputery;Tablety i akcesoria;Tablety,1816.97
2861,6769,2021-09-30 23:45:23,298,1277,VIEW_PRODUCT,0,NaN,39,9,3,2021-09-30,Apple iPad mini 64GB 4G,Komputery;Tablety i akcesoria;Tablety,2317.02


In [108]:
def get_user_id_from_session(session):
    sample_user_id = session['user_id'].iloc[0]
    for user_id in session['user_id']:
        if sample_user_id != user_id:
            raise Exception("How it is even possible")
    return sample_user_id

def get_session_id_from_session(session):
    sample_session_id = session['user_id'].iloc[0]
    for session_id in session['user_id']:
        if sample_session_id != session_id:
            raise Exception("How it is even possible")
    return sample_session_id


In [109]:
def check_if_user_bought_something(session):
    for event_type in session['event_type']:
        if event_type == 'BUY_PRODUCT':
            return True
    return False

In [110]:
def get_session_information(session):
    d = {
        'session_id': get_session_id_from_session(session),
        'beginning': [min(session['timestamp'])],
        'end': [max(session['timestamp'])],
        'user_id' : get_user_id_from_session(session),
        'bought_product': check_if_user_bought_something(session)
    }
    df = pd.DataFrame(data=d)
    return df.set_index('session_id')

In [111]:
def extract_session_batch(sessions_data):
    sessions = []
    for session_id in sessions_data['session_id'].unique():
        sessions.append(get_session_information(sessions_data[sessions_data['session_id'] == session_id]))
    enriched_session_batch = pd.concat(sessions)
    return enriched_session_batch

In [103]:
extracted_session_batch_one_month = extract_session_batch(sample_month_data)

In [104]:
# extracted_session_batch[extracted_session_batch.bought_product == True]

In [105]:
extracted_session_batch_one_year = extract_session_batch(sessions_data)

In [112]:
extracted_session_batch_one_year

,beginning,end,user_id,bought_product
session_id,,,,
139,2021-01-08 17:29:37,2021-01-08 17:35:32,139,True
139,2021-01-10 05:56:20,2021-01-10 06:23:24,139,True
242,2021-01-10 12:44:13,2021-01-10 13:06:59,242,False
108,2021-01-11 00:25:44,2021-01-11 00:28:09,108,True
143,2021-01-12 08:48:51,2021-01-12 08:51:03,143,True
...,...,...,...,...
128,2021-12-10 09:15:24,2021-12-10 09:19:52,128,True
287,2021-12-10 09:21:54,2021-12-10 09:55:10,287,True
300,2021-12-10 09:39:24,2021-12-10 09:39:24,300,False


# Ekstrakcja userów

In [115]:
def get_user_information(user_session_data):
    d = {
        'user_id': [get_user_id_from_session(user_session_data)],
        'expenses': [user_session_data[user_session_data['event_type'] == "BUY_PRODUCT"]['price'].sum()],
        'products_bought': [len(user_session_data[user_session_data['event_type'] == "BUY_PRODUCT"])],
        'events_number': [len(user_session_data)]
    }
    df = pd.DataFrame(data=d)
    return df.set_index('user_id')

In [116]:
def extract_users_data(sessions_data, users_data, products_data):
    enriched_sessions_data = pd.merge(sessions_data, products_data, on="product_id").sort_values(by=['timestamp'])
    extracted_users = []
    for user_id in enriched_sessions_data['user_id'].unique():
        extracted_users.append(get_user_information(enriched_sessions_data[enriched_sessions_data['user_id'] == user_id]))
    enriched_users_data = pd.concat(extracted_users)
    return pd.merge(enriched_users_data, users_data, on="user_id").drop(columns=['name', 'street'])
    # print(len(sessions_data.user_id.unique()))

In [118]:
extract_users_data(sessions_data, users_data, products_data)

,user_id,expenses,products_bought,events_number,city
0,139,23400.85,49,256,Kraków
1,242,44677.14,60,331,Radom
2,108,16312.03,23,130,Radom
3,143,7273.05,16,86,Kraków
4,140,33412.55,44,318,Poznań
...,...,...,...,...,...
195,289,0.00,0,7,Warszawa
196,225,109.00,1,5,Warszawa
197,152,0.00,0,5,Poznań
198,162,78.96,2,6,Szczecin


In [33]:
enriched_sessions_data = pd.merge(sessions_data, products_data, on="product_id").sort_values(by=['timestamp'])
user_session_data = enriched_sessions_data[enriched_sessions_data['user_id'] == 137]
user_session_data

,session_id,timestamp,user_id,product_id,event_type,offered_discount,purchase_id,timestamp_week,timestamp_month,timestamp_quarter,timestamp_date,product_name,category_path,price
16718,1535,2021-06-24 20:30:32,137,1048,VIEW_PRODUCT,15,NaN,25,6,2,2021-06-24,Gra o tron (PC),Gry i konsole;Gry komputerowe,63.49
3600,1535,2021-06-24 20:32:27,137,1050,VIEW_PRODUCT,15,NaN,25,6,2,2021-06-24,Bioshock 2 (PC),Gry i konsole;Gry komputerowe,37.90
5232,1535,2021-06-24 20:37:12,137,1054,VIEW_PRODUCT,15,NaN,25,6,2,2021-06-24,Call of Duty 2 (PC),Gry i konsole;Gry komputerowe,32.99
16991,1535,2021-06-24 20:41:01,137,1056,VIEW_PRODUCT,15,NaN,25,6,2,2021-06-24,Call of Duty Black Ops (PC),Gry i konsole;Gry komputerowe,29.99
4429,1535,2021-06-24 20:41:12,137,1053,VIEW_PRODUCT,15,NaN,25,6,2,2021-06-24,Anno 2070 (PC),Gry i konsole;Gry komputerowe,42.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4891,1547,2021-12-07 04:38:35,137,1051,VIEW_PRODUCT,0,NaN,49,12,4,2021-12-07,Spec Ops The Line (PC),Gry i konsole;Gry komputerowe,76.90
4061,1547,2021-12-07 04:43:20,137,1052,VIEW_PRODUCT,0,NaN,49,12,4,2021-12-07,Duke Nukem Forever (PC),Gry i konsole;Gry komputerowe,78.90
4062,1547,2021-12-07 04:46:16,137,1052,BUY_PRODUCT,0,20785.0,49,12,4,2021-12-07,Duke Nukem Forever (PC),Gry i konsole;Gry komputerowe,78.90
19441,1521,2021-12-07 09:16:56,137,1283,VIEW_PRODUCT,20,NaN,49,12,4,2021-12-07,Okulary 3D PHILIPS PTA436/00,Sprzęt RTV;Video;Telewizory i akcesoria;Okular...,99.99


In [119]:
get_user_information(user_session_data)

,expenses,products_bought,events_number
user_id,,,
137,23662.84,35,179


In [120]:
extract_users_data(sessions_data, users_data, products_data)

,user_id,expenses,products_bought,events_number,city
0,139,23400.85,49,256,Kraków
1,242,44677.14,60,331,Radom
2,108,16312.03,23,130,Radom
3,143,7273.05,16,86,Kraków
4,140,33412.55,44,318,Poznań
...,...,...,...,...,...
195,289,0.00,0,7,Warszawa
196,225,109.00,1,5,Warszawa
197,152,0.00,0,5,Poznań
198,162,78.96,2,6,Szczecin


# Archiwum
Funkcje niegdyś przydatne, teraz już niepotrzebne, ale szkoda wyrzucać

In [106]:
def find_returned_users(extracted_sessions_data):
    user_counts = extracted_sessions_data['user_id'].value_counts()
    return user_counts[user_counts>=2].index

In [107]:
def find_never_returned_users(extracted_sessions_data):
    user_counts = extracted_sessions_data['user_id'].value_counts()
    return user_counts[user_counts<2].index